In [105]:
import pandas as pd
from datetime import datetime

file_path = '/Users/HM/Desktop/Code/FOIL_DA_Task-10-2024/data/Online_Retail.xlsx'
df = pd.read_excel(file_path)


In [106]:

def load_data(file_path):

    df = pd.read_excel(file_path)

    df = df.sort_values(by='InvoiceDate').reset_index(drop=True)

    return df.shape, df.head(5), df.info()

In [107]:
print(load_data(file_path))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB
((541909, 8),   InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    5


A large number of CustomerIDs are missing, and will be filled in using '-1'.

Direct visual inspection of the spreadsheet shows that some items seem to have been mislabeled as other items.

Entries in the Description column are in combination upper- and lower-case, while item names are entirely in upper-case. This does not need to be changed. 

In [108]:
def empty_values(file_path):
    
    
    df = pd.read_excel(file_path)

    cols = ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']
    
    empty_values_count = {}
    
    for col in cols:
        if df[col].dtype == 'object':
            empty_values = df[df[col].isnull() | (df[col].str.strip() == "")]
        else:
            empty_values = df[df[col].isnull()]

        empty_values_count[col] = len(empty_values)

        if len(empty_values) > 0:
            print(f"Column {col} has {len(empty_values)} missing or empty values")
        else:
            print(f"No empty or missing values found in {col} column")
            
    return empty_values_count

print(empty_values(file_path))

No empty or missing values found in InvoiceNo column
No empty or missing values found in StockCode column
Column Description has 1454 missing or empty values
No empty or missing values found in Quantity column
No empty or missing values found in InvoiceDate column
No empty or missing values found in UnitPrice column
Column CustomerID has 135080 missing or empty values
No empty or missing values found in Country column
{'InvoiceNo': 0, 'StockCode': 0, 'Description': 1454, 'Quantity': 0, 'InvoiceDate': 0, 'UnitPrice': 0, 'CustomerID': 135080, 'Country': 0}


Filling empty Customer IDs and Descriptions with suitable values

In [109]:
def fill_missing_cust_id(file_path):

    df = pd.read_excel(file_path)
    prior_missing_cust_ids= df['CustomerID'].isnull().sum()
    df['CustomerID'] = df['CustomerID'].fillna("Missing ID")

    post_missing_cust_ids = df['CustomerID'].isnull().sum()

    return prior_missing_cust_ids, post_missing_cust_ids

In [110]:
def fill_missing_description(file_path):
    
    df = pd.read_excel(file_path)
    
    prior_missing_descriptions = df['Description'].isnull().sum()
    
    df['Description'] = df['Description'].fillna("Missing Description")
    
    post_missing_descriptions = df['Description'].isnull().sum()
    
    df.to_excel(file_path, index=False)
    return prior_missing_descriptions, post_missing_descriptions

In [111]:
for col in ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']:
    print(f"{df[col].isnull().mean() * 100}% of {col} is missing")

0.0% of InvoiceNo is missing
0.0% of StockCode is missing
0.2683107311375157% of Description is missing
0.0% of Quantity is missing
0.0% of InvoiceDate is missing
0.0% of UnitPrice is missing
24.926694334288598% of CustomerID is missing
0.0% of Country is missing


We see there are two columns where values are missing, so we will need to deal with those. We also need to check those values that do not make any sense e.g. negative values for Quantity or UnitPrice.

First, however, we need to drop duplicate rows to prevent double-counting.

In [112]:
def drop_duplicates(file_path):

    df = pd.read_excel(file_path)
    df = df.drop_duplicates()
    
    return df

Next we deal with non-positive prices.

In [125]:
def impute_unit_price(df):
    mask = df['UnitPrice'] <= 0
    for stock_code in df.loc[mask, 'StockCode'].unique():
        positive_prices = df[(df['StockCode'] == stock_code) & (df['UnitPrice'] > 0)]['UnitPrice']
        if not positive_prices.empty:
            mean_positive_price = positive_prices.mean()
            df.loc[(df['StockCode'] == stock_code) & mask, 'UnitPrice'] = mean_positive_price
            
    return df


In [126]:
negative_or_zero_price_percentage = (df['UnitPrice'] <= 0).mean() * 100
print(f"{negative_or_zero_price_percentage:.2f}% of UnitPrice values are zero or negative")


0.46% of UnitPrice values are zero or negative


In [127]:
cleaned_df = impute_unit_price(df)
new_negative_or_zero_price_percentage = (cleaned_df['UnitPrice'] <= 0).mean() * 100
print(f"After imputation, {new_negative_or_zero_price_percentage:.2f}% of UnitPrice values are zero or negative")

After imputation, 0.02% of UnitPrice values are zero or negative


After imputation, we are left with a very small number of negative or zero UnitPrices. 
Because of the impossibility of finding out what these prices should be, we will drop them.


In [128]:
negative_quantity = (df["Quantity"] <= 0).mean() * 100
print(f"{negative_quantity:.2f}% of Quantity values are zero or negative")

1.96% of Quantity values are zero or negative


Because the cases where Quantity is zero or negative deal with adjustments or outflows, we will ignore that data when looking for the top-selling data and average consumer statistics. 

In [116]:
df["Country"].value_counts(normalize=True).head(10) * 100

Country
United Kingdom    91.431956
Germany            1.752139
France             1.579047
EIRE               1.512431
Spain              0.467422
Netherlands        0.437527
Belgium            0.381798
Switzerland        0.369435
Portugal           0.280305
Australia          0.232327
Name: proportion, dtype: float64

The UK accounts for almost 92% of all transactions, with the rest being made up of neighbouring EU countries. 

In [117]:
print(f"The first transaction was made at {df['InvoiceDate'].min()}")
print(f"The latest transaction was made at {df['InvoiceDate'].max()}")
print(f"There were a total of {df['InvoiceNo'].nunique()} transactions")

The first transaction was made at 2010-12-01 08:26:00
The latest transaction was made at 2011-12-09 12:50:00
There were a total of 25900 transactions
